# Machine Learning Model

## Import csv

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../../cyberbullying/data/data.csv')
data.head()

,text,target
0,this is not creative those are the dictionar...,0.0
1,the term standard model is itself less npov...,0.0
2,true or false the situation as of march was...,0.0
3,next maybe you could work on being less conde...,0.0
4,this page will need disambiguation,0.0


In [3]:
data.shape

(230155, 2)

## Data Cleaning 2

### Stopwords

In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

def remove_stop_words(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    text = [w for w in word_tokens if not w in stop_words]
    text= ' '.join(text)
    return text


In [5]:
data['text'] = data['text'].apply(remove_stop_words)
data.head()


,text,target
0,creative dictionary definitions terms insuranc...,0.0
1,term standard model less npov think wed prefer...,0.0
2,true false situation march saudi proposal land...,0.0
3,next maybe could work less condescending sugge...,0.0
4,page need disambiguation,0.0


### Lemmatize

In [6]:
data.head()

,text,target
0,creative dictionary definitions terms insuranc...,0.0
1,term standard model less npov think wed prefer...,0.0
2,true false situation march saudi proposal land...,0.0
3,next maybe could work less condescending sugge...,0.0
4,page need disambiguation,0.0


In [7]:
from nltk.stem import WordNetLemmatizer

def lematize_words(text):
    token_words=word_tokenize(text)
    token_words
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in token_words]
    text = lemmatized
    text= ' '.join(text)
    return text


In [8]:
data['text'] = data['text'].apply(lematize_words)
data.head()

,text,target
0,creative dictionary definition term insurance ...,0.0
1,term standard model le npov think wed prefer n...,0.0
2,true false situation march saudi proposal land...,0.0
3,next maybe could work le condescending suggest...,0.0
4,page need disambiguation,0.0


### Stemmer

In [9]:
from nltk.stem.porter import PorterStemmer
def stemmer_word(text):
    token_words=word_tokenize(text)
    token_words
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word) for word in token_words]
    text = stemmed
    text= ' '.join(text)
    return text

In [11]:
data['text'] = data['text'].apply(stemmer_word)
data.head()

,text,target
0,creativ dictionari definit term insur ensur pr...,0.0
1,term standard model le npov think wed prefer n...,0.0
2,true fals situat march saudi propos land peac ...,0.0
3,next mayb could work le condescend suggest rea...,0.0
4,page need disambigu,0.0
